# Syncing Audiobooks and  Ebooks with Speech Recognition

I'm a huge fan of audiobooks because of how faster it is to listen to an audiobook than actually reading it. However, I've always felt that I absorbed the material much better when I put the time to sit down and read the book. Some recent studies also suggest the reality of this phenomenon ([see here](https://time.com/5388681/audiobooks-reading-books/)). Therefore, the concept of reading and listening at the same time has been appealing. 

In the past few years, the only way to do this would be to get the audiobook and the ebook separately and to follow along with the book while the audio played in the background. But a much better way to smooth this experience would be to have the ebook autoscroll to the speed of the spoken word. Audible introduced their product called Immersive Reading in 2012, but it only became popular in the last few years and its price is not great.

![audible](../images/audible_smaller.jpg) 

![audible2](../images/imersive.jpg) 



To the best of my knowledge, there are currently no free softwares that offer this service and so I started wondering how hard could it be.

## Preliminary Brainstorm

The problem consists in finding, for a given portion of the audiobook, the location of the corresponding text in the book. 

I was very optimistic that I could solve a problem like this a priori, because we do not need our model to work very well at the word level (as it is usual in speech processing), but we only need to get roughly right which sentence the audio is pointg to and then scroll to that paragraph. It was also clear that the span of sentences to be looked for at each time is not the whole book since, in the first seconds of the audiobook, the corresponding text will also be in the first sentences of the file.

So, our model should receive a chunk of the audiobook and a set of chuncks of text as input and it should predict which chunk of text best fits the audio.

Some of the first few ideas for solutions and possible obstacles from the initial brainstorm consisted in:
* Splitting the audio in equal time spans vs detecting the limits of each sentence/ word.
* Converting the audio chuncks to spectograms and use that to train a neural network such as:
 * A CNN 
 * An RNN
 * A Siamese model
* Using a pretrained model to predict the text for each audio and use that as the input of a simpler model.

The last one was obviously the first one to try, even though the junior data scientist instinct kicked in by making me dream with different NN architectures for solving a possibly simple model.

## Dataset

In the beginning, it was not obvious if the solution should be applied at the word level or the sentence level. So, after exploring suitable datasets (with text files and the corresponding audio), I found some sentence-audio datasets, but no word level ones. The first choice was The World English Bible dataset, which is composed by a transcript of the whole bible with each verse marked and an .wav file for each one:

![audible2](../images/dataset.PNG) 

The dataset can be found in Kaggle (https://www.kaggle.com/bryanpark/the-world-english-bible-speech-dataset?select=transcript.txt).

## Speech Processing

A few months ago, when I first started toying around with these concepts, the idea was to use my microphone to test diferent speeds and voices for the same text. However, the standard python library, **Speech Recognition**, was too slow for this real-time text recognition, so I tried the offline Sphinx engine from the same library.

This first proof of concept consists in using Sphinx to transcribe each audio file and then choose, out of a few candidate sentences near the last prediction, the most likely one.

The transcription of the audio file is done with the following simple function:

In [1]:
import speech_recognition as sr

def speech_text(AUDIO_FILE):
    # use the audio file as the audio source
    r = sr.Recognizer()
    with sr.AudioFile(AUDIO_FILE) as source:
        audio = r.record(source, duration=5)  # read the entire audio file

    # recognize speech using Sphinx
    try:
        out = r.recognize_sphinx(audio)
        return out

    except sr.UnknownValueError:
        print("Sphinx could not understand audio")
        return ""
    except sr.RequestError as e:
        print("Sphinx error; {0}".format(e))
        return ""

The **first 4 verses** of the Bible are:

>In the beginning God created the heavens and the earth.

>Now the earth was formless and empty. Darkness was on the surface of the deep. God's Spirit was hovering over the surface of the water.

>God said, "Let there be light," and there was light.

>God saw the light, and saw that it was good. God divided the light from the darkness.



Now, for the first verse, as an example:

In [31]:
genesis_1_1 = 'audio/'+ 'Genesis/Genesis_1-1' +'.wav'
print("Predicted:")
speech_text(genesis_1_1)

Predicted:


'in the beginning god created the heavens and the year and'

We can see that the prediction, in this case, was quite good, but we still need to choose a metric to calculate the similarity of the predicted sentence and each of the candidate original sentences.